In [1]:
import sys
from pathlib import Path
sys.path.append("/kb/module/berdl")
from berdl.genome_paths import GenomePaths
from berdl.prep_genome_set import BERDLPreGenome
from cobrakbase import KBaseAPI

modelseedpy 0.4.2
cobrakbase 0.4.0


In [72]:
import polars as pl
pl.thread_pool_size()

336

In [2]:
from berdl.query.query_pangenome_local import QueryPangenomeLocal
from berdl.query.query_genome_local import QueryGenomeLocal
from berdl.pangenome.pangenome import BERDLPangenome
from berdl.pangenome.paths_pangenome import PathsPangenome

In [3]:
scratch = '/kb/module/work/tmp'
root_pangenome = Path(scratch) / 'pangenome' / 'GB_GCA_021307345.1'
paths = PathsPangenome(root=root_pangenome)

In [7]:
query_pg = QueryPangenomeLocal('/data/reference_data/berdl_db/ke-pangenomes')
query_g = QueryGenomeLocal()

In [9]:
from modelseedpy.core.msgenome import MSFeature, MSGenome
from berdl.pangenome.paths_pangenome import PathsPangenome


class BERDLPangenome:

    def __init__(self, query_pg, query_g, paths: PathsPangenome):
        self.pg = query_pg  # pan-genome query api
        self.query_g = query_g
        self.paths = paths.ensure()

    def run(self, genome_id):
        clade_id = self.pg.get_member_representative(genome_id)
        clade_members = self.pg.get_clade_members(clade_id)
        clade_gene_clusters = self.pg.get_clade_gene_clusters(clade_id)
        clade_cluster_ids = set(clade_gene_clusters['gene_cluster_id'])
        df_gene_genecluster = self.pg.get_clusters_members(clade_cluster_ids)
        d_gene_to_cluster = {o[0]: o[1] for o in df_gene_genecluster.iter_rows()}
        d_cluster_to_genes = {}
        for k, v in d_gene_to_cluster.items():
            if v not in d_cluster_to_genes:
                d_cluster_to_genes[v] = set()
            d_cluster_to_genes[v].add(k)

        # get clade member faa and fna
        members = {}
        for row in clade_members.rows(named=True):
            member_id = row['genome_id']
            members[member_id] = self.query_g.get_genome_features(member_id)

        # build master protein user_genome + pangenome

        u_proteins = {}
        for k, g in members.items():
            print(k, len(g.features))
            for feature in g.features:
                _parts = feature.description.split(' ')
                h = _parts[4]
                u_proteins[h] = MSFeature(h, feature.seq)

        genome_master_faa = MSGenome()
        genome_master_faa.add_features(list(u_proteins.values()))
        genome_master_faa.to_fasta(str(self.paths.out_master_faa_pangenome_members))

        #  collect user genome and add to u_proteins
        

        #  collect phenotype and fitness
        genome_master_faa_fitness = MSGenome.from_fasta(str(self.paths.ref_master_faa_protein_fitness))
        for feature in genome_master_faa_fitness:
            if feature.id not in u_proteins:
                u_proteins[feature.id] = MSFeature(feature.id, feature.seq)
        genome_master_faa_phenotype = MSGenome.from_fasta(str(self.paths.ref_master_faa_protein_phenotype))
        for feature in genome_master_faa_phenotype:
            if feature.id not in u_proteins:
                u_proteins[feature.id] = MSFeature(feature.id, feature.seq)

        # rebuild master faa genome with proteins from
        #  user / pangenome / fitness / phenotypes
        genome_master_faa = MSGenome()
        genome_master_faa.add_features(list(u_proteins.values()))
        genome_master_faa.to_fasta(str(self.paths.out_master_faa))


In [41]:
import polars as pl
from pathlib import Path

paths = sorted(Path('/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes').glob("block_*/*name*.parquet"))

for p in paths:
    n = pl.scan_parquet(str(p)).select(pl.len()).collect().item()
    print(p)

/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_0/feature_name.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_0/name.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_1/name.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_1/name_feature.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_2/name.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_2/name_feature.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_3/name.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_3/name_feature.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_4/name.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_4/name_feature.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_5/name.parquet
/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_5/name_feature.parquet


In [36]:
print(query_g.ldf_name.explain(streaming=True))

Parquet SCAN [/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_0/feature_name.parquet, ... 11 other sources]
PROJECT */4 COLUMNS
ESTIMATED ROWS: 2281548108


/tmp/ipykernel_20155/1651664276.py:1: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  print(query_g.ldf_name.explain(streaming=True))


In [68]:
ldf_name = pl.scan_parquet(
    [
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_0/name.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_1/name.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_2/name.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_3/name.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_4/name.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_5/name.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_0/feature_name.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_1/name_feature.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_2/name_feature.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_3/name_feature.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_4/name_feature.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_5/name_feature.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/name.parquet',
        '/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/name_feature.parquet',
    ]
)
ldf_name.explain()

'Parquet SCAN [/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes/block_0/name.parquet, ... 13 other sources]\nPROJECT */4 COLUMNS\nESTIMATED ROWS: 61191284'

In [69]:
df = ldf_name.filter(pl.col("name") == 'GB_GCA_019839465.1').select("entity_id").collect()
df

entity_id
str
"""CDM:ccol-b754b396-c86c-41e4-ad…"
"""CDM:ccol-b754b396-c86c-41e4-ad…"


{'CDM:ccol-b754b396-c86c-41e4-ada9-c416a4761802'}

In [28]:
query_g.get_cdm_genome_id_by_name('GB_GCA_019459165.1')

{'entity_id': 'CDM:ccol-9034b761-4f23-4bc4-ad6b-54e7ec40366a',
 'name': 'GB_GCA_019459165.1',
 'description': 'gtdb_r214',
 'source': 'ke-pangenomes'}

In [13]:
import json
with open(paths.genome_prep_clade_data, 'r') as fh:
    clade_assignment = json.load(fh)

In [23]:
selected_clade_member_id = 'GB_GCA_021307345.1'

In [25]:
{k for k, v in clade_assignment.items() if v == selected_clade_member_id}

{'Escherichia_coli_K-12_MG1655'}

In [10]:
berld_pangenome = BERDLPangenome(query_pg, query_g, paths)

In [11]:
member_id = 'GB_GCA_021307345.1'
berld_pangenome.run(member_id)

GB_GCA_019459165.1 5983
RS_GCF_001633105.1 4955


TypeError: 'MSGenome' object is not iterable

In [7]:
clade_id

's__Acidovorax_sp001633105--RS_GCF_001633105.1'

In [8]:
members = {}
for row in clade_members.rows(named=True):
    member_id = row['genome_id']
    members[member_id] = berld_pangenome.query_g.get_genome_features(member_id)

In [22]:
from modelseedpy.core.msgenome import MSFeature, MSGenome

In [23]:
u_proteins = {}
for k, g in members.items():
    print(k, len(g.features))
    for feature in g.features:
        _parts = feature.description.split(' ')
        h = _parts[4]
        u_proteins[h] = MSFeature(h, feature.seq)
len(u_proteins)

GB_GCA_019459165.1 5983
RS_GCF_001633105.1 4955


9997

'2861cd6923265c67fd04b835d48aa89c383f5cae645e53c14ee338e3cc4a56f1'

In [10]:
from berdl.hash_seq import ProteinSequence

In [23]:
clade_member_ids = {t[0] for t in clade_members.rows()}

In [24]:
clade_member_ids

{'GB_GCA_019459165.1', 'RS_GCF_001633105.1'}

In [ ]:
pl.scan_parquet(f'{self.root_reference}/block_*/name.parquet')

In [ ]:
CDM:ccol-9034b761-4f23-4bc4-ad6b-54e7ec40366a

In [132]:
import polars as pl
from pathlib import Path
from berdl.query.query_genome import QueryGenomeABC


class QueryGenomeLocal(QueryGenomeABC):

    def __init__(self):
        self.root_reference = Path('/data/reference_data/berdl_db/cdm_genomes/ke-pangenomes')
        self.ldf_name = pl.scan_parquet(f'{self.root_reference}/block_*/*name*.parquet')
        #self.ldf_name_feature = pl.scan_parquet(f'{self.root_reference}/block_*/name_feature.parquet')
        self.ldf_ccol_x_protein = pl.scan_parquet(f'{self.root_reference}/block_*/contig_collection_x_protein.parquet')
        self.ldf_ccol_x_feature = pl.scan_parquet(f'{self.root_reference}/block_*/contig_collection_x_feature.parquet')
        self.ldf_contig_x_feature = pl.scan_parquet(f'{self.root_reference}/block_*/contig_x_feature.parquet')
        self.ldf_protein = pl.scan_parquet(f'{self.root_reference}/block_*/protein.parquet')
        self.ldf_feature = pl.scan_parquet(f'{self.root_reference}/block_*/feature.parquet')
        self.ldf_feature_x_protein = pl.scan_parquet(f'{self.root_reference}/block_*/feature_x_protein.parquet')

    def get_cdm_genome_id_by_name(self, name: str):
        df = self.ldf_name.filter(pl.col("name") == name).collect()
        if df.height == 1:
            return df.row(0, named=True)
        return None

    def get_genome_features(self, genome_id: str):
        res = self.get_cdm_genome_id_by_name(genome_id)
        if res:
            ccol_id = res['entity_id']
            df = self.ldf_ccol_x_feature.filter(pl.col("contig_collection_id") == ccol_id).select(
                pl.col("feature_id")).collect()
            cdm_feat_ids = {o[0] for o in df.rows()}

            df_features = self.ldf_feature.filter(pl.col("feature_id").is_in(cdm_feat_ids)).select(
                ["feature_id", "start", "end", "type"]).collect()

            df_feature_to_contig = self.ldf_contig_x_feature.filter(
                pl.col("feature_id").is_in(cdm_feat_ids)).collect()
            d_feature_to_contig = {row['feature_id']: row['contig_id'] for row in df_feature_to_contig.rows(named=True)}

            df_feature_to_protein = self.ldf_feature_x_protein.filter(
                pl.col("feature_id").is_in(cdm_feat_ids)).collect()
            d_feature_to_protein = {row['feature_id']: row['protein_id'] for row in
                                    df_feature_to_protein.rows(named=True)}

            cdm_prot_ids = set(d_feature_to_protein.values())
            df_proteins = self.ldf_protein.filter(pl.col("protein_id").is_in(cdm_prot_ids)).collect()
            cdm_proteins = {}
            for row in df_proteins.rows(named=True):
                cdm_proteins[row['protein_id']] = row

            cntg_ids = set(d_feature_to_contig.values())
            df_contig_names = self.ldf_name.filter(pl.col("entity_id").is_in(cntg_ids)).select(
                ["entity_id", "name"]).collect()
            d_cntg_id_to_name = {row['entity_id']: row['name'] for row in df_contig_names.rows(named=True)}

            df_names = self.ldf_name.filter(pl.col("entity_id").is_in(cdm_feat_ids)).select(
                ["entity_id", "name"]).collect()

            d_names = {row['entity_id']: row['name'] for row in df_names.rows(named=True)}

            from modelseedpy.core.msgenome import MSFeature, MSGenome
            features = {}
            for row in df_features.rows(named=True):
                start = row['start']
                end = row['end']
                feature_type = row['type']
                cdm_feature_id = row['feature_id']
                cdm_protein_id = d_feature_to_protein[cdm_feature_id]
                cdm_protein = cdm_proteins[cdm_protein_id]
                protein_hash = cdm_protein['hash']
                sequence = cdm_protein['sequence']
                cdm_contig_id = d_feature_to_contig[cdm_feature_id]
                contig_name = d_cntg_id_to_name[cdm_contig_id]
                feature_to_kpg_id = d_names.get(cdm_feature_id, cdm_feature_id)
                desc = f'{contig_name} {start} {end} {feature_type} {protein_hash} {cdm_contig_id} {cdm_feature_id} {cdm_protein_id}'
                genome_feature = MSFeature(feature_to_kpg_id, sequence, description=desc)
                features[genome_feature.id] = genome_feature

            genome = MSGenome()
            genome.add_features(list(features.values()))
            return genome

        else:
            return None

    def get_genome_contigs(self, genome_id: str):
        """Return all gene clusters for a given clade."""
        raise NotImplementedError


In [133]:
query_genome = QueryGenomeLocal()

In [134]:
genome_member = query_genome.get_genome_features('GB_GCA_019459165.1')

In [140]:
for f in genome_member.features:
    print(f.id, f.description)
    break

JAIBEQ010000430.1_1 JAIBEQ010000430.1 1 2145 CDS 1476cb0d2f1f6946951c4bf697ae4aaa7a09cc669ac3350465a5baa45eacc778 CDM:cntg-0652c309-15ab-47fc-9c7a-fba4528b5d96 CDM:feat-7789e6da-eab5-46c7-9107-7add5ae62185 CDM:prot-de23b6c3-c0e3-4df9-95e0-0c6e6bd10e6e


In [77]:
df = query_genome.ldf_ccol_x_feature.filter(pl.col("contig_collection_id") == ccol_id).select(pl.col("feature_id")).collect()
cdm_feat_ids = {o[0] for o in df.rows()}
df_features = query_genome.ldf_feature.filter(pl.col("feature_id").is_in(cdm_feat_ids)).select(["feature_id", "start", "end", "type"]).collect()

In [125]:
from modelseedpy.core.msgenome import MSFeature, MSFeature
features = {}
for row in df_features.rows(named=True):
    start = row['start']
    end = row['end']
    feature_type = row['type']
    cdm_feature_id = row['feature_id']
    cdm_protein_id = d_feature_to_protein[cdm_feature_id]
    cdm_protein = cdm_proteins[cdm_protein_id]
    contig_id = d_feature_to_contig[cdm_feature_id]
    protein_hash = cdm_protein['hash']
    sequence = cdm_protein['sequence']
    feature_to_kpg_id = d_names.get(cdm_feature_id, cdm_feature_id)
    desc = f'{start} {end} {feature_type}'
    MSFeature(feature_to_kpg_id, sequence, description=)
    break

In [131]:
df_contig_names

entity_id,name
str,str
"""CDM:cntg-0652c309-15ab-47fc-9c…","""JAIBEQ010000430.1"""
"""CDM:cntg-77920f40-360f-4b30-b7…","""JAIBEQ010000456.1"""
"""CDM:cntg-35e9d20a-1730-471b-8e…","""JAIBEQ010000495.1"""
"""CDM:cntg-a142609d-adf6-4587-8c…","""JAIBEQ010000514.1"""
"""CDM:cntg-3cecc5bf-5c5b-440c-b6…","""JAIBEQ010000569.1"""
…,…
"""CDM:cntg-e6a45a6a-24ed-474f-85…","""JAIBEQ010000898.1"""
"""CDM:cntg-c0a8dd32-d692-440c-93…","""JAIBEQ010000903.1"""
"""CDM:cntg-184350e7-9c4c-470d-a6…","""JAIBEQ010000916.1"""


In [121]:
d_names = {row['entity_id']: row['name'] for row in df_names.rows(named=True)}

In [116]:
df_names

entity_id,name
str,str
"""CDM:feat-7789e6da-eab5-46c7-91…","""JAIBEQ010000430.1_1"""
"""CDM:feat-8d9e7d7e-6bdc-4bb9-81…","""JAIBEQ010000430.1_2"""
"""CDM:feat-7923a96d-9def-47cd-80…","""JAIBEQ010000430.1_3"""
"""CDM:feat-ff7fa704-472b-4575-aa…","""JAIBEQ010000456.1_1"""
"""CDM:feat-ad7617c8-9dbe-4004-94…","""JAIBEQ010000456.1_2"""
…,…
"""CDM:feat-7e1ce321-3940-4130-85…","""JAIBEQ010000183.1_6"""
"""CDM:feat-3b38b767-d1ec-4222-a9…","""JAIBEQ010000183.1_7"""
"""CDM:feat-9983306d-f095-4b65-b1…","""JAIBEQ010000183.1_8"""


In [101]:
d_feature_to_protein[cdm_feature_id]

'CDM:prot-de23b6c3-c0e3-4df9-95e0-0c6e6bd10e6e'

In [106]:
cdm_protein['hash']

{'protein_id': 'CDM:prot-de23b6c3-c0e3-4df9-95e0-0c6e6bd10e6e',
 'hash': '1476cb0d2f1f6946951c4bf697ae4aaa7a09cc669ac3350465a5baa45eacc778',
 'description': None,
 'evidence_for_existence': None,
 'length': 714,
 'sequence': 'TWRASIIPLLAVPVSVVGTFAVLHLLGFSINALSLFGLVLAIGIVVDDAIVVVENVERNIEAGLTPREATYRAMREVSGPIIAIALVLVAVFVPLAFISGLTGQFYKQFAVTIAISTVISAINSLTLSPALSALLLKGHDEPKDALTRGMDKVFGGLFRGFNRLFHRGSEAYSGGVKRVIGRKALMFVIYLALVGATMGLFKIVPGGFVPAQDKQYLIGFAQLPDGATLDRTENVIRRMGEIMKENPNVEDAIAFPGLSINGFTNSSNSGIVFATLKPFAERTRADQSGGAVAGQLNQSFGSIQDAFIAMFPPPPVAGLGTTGGFKLQIEDRASLGYEAMDNAVKAFMGKAYQTPELAGLFTSWQVNVPQLYANIDRTKARQLGVPVTDIFDTLQIYLGSLYANDFNQFGRTYSVRVQADAAYRARAEDVGLLKVRSATGEMVPLSALMKMEPSFGPERAMRYNGYLAADVNGGPAPGYSSGQAQAAIERIAKETLPQGITFEWTELTYQEILAGNSAVLVFPLAILLVFLVLAAQYESLTLPIAIILIVPMGILAAMTGVWLTKGDNNVFTQIGLIVLVGLSAKNAILIVEFARELEFAGRTPVQAAIEASRLRLRPILMTSLAFVMGVLPLVLSTGAGAEMRSAMGVAVFAGMIGVTAFGLFLTPVFYVLLRKLAGNRPLVEHGAHVAPISHAPGTGGTAHPVLAAPRKEHE'}

In [91]:
df_feature_to_protein

feature_id,protein_id
str,str
"""CDM:feat-7789e6da-eab5-46c7-91…","""CDM:prot-de23b6c3-c0e3-4df9-95…"
"""CDM:feat-8d9e7d7e-6bdc-4bb9-81…","""CDM:prot-58ae8786-cc2f-4c07-b4…"
"""CDM:feat-7923a96d-9def-47cd-80…","""CDM:prot-fd7bc475-fd90-4ab4-b1…"
"""CDM:feat-ff7fa704-472b-4575-aa…","""CDM:prot-9d28af3f-292d-48b1-8a…"
"""CDM:feat-ad7617c8-9dbe-4004-94…","""CDM:prot-2fdf32aa-ffbf-4e99-93…"
…,…
"""CDM:feat-7e1ce321-3940-4130-85…","""CDM:prot-52202d52-9bca-4b86-83…"
"""CDM:feat-3b38b767-d1ec-4222-a9…","""CDM:prot-cf6b84ca-5214-4d96-b7…"
"""CDM:feat-9983306d-f095-4b65-b1…","""CDM:prot-5732a142-f44a-4786-be…"


In [34]:
df = ldf_ccol_x_protein.filter(pl.col("contig_collection_id") == ccol_id).select(pl.col("protein_id")).collect()

In [39]:
cdm_prot_ids = {o[0] for o in df.rows()}

In [43]:
df_proteins = ldf_protein.filter(pl.col("protein_id").is_in(cdm_prot_ids)).collect()

In [58]:
df = ldf_ccol_x_feature.filter(pl.col("contig_collection_id") == ccol_id).select(pl.col("feature_id")).collect()

In [60]:
cdm_feat_ids = {o[0] for o in df.rows()}

In [62]:
len(cdm_feat_ids)

5983

In [108]:
query_genome.ldf_name.collect_schema()

Schema([('entity_id', String),
        ('name', String),
        ('description', String),
        ('source', String)])

In [68]:
df_features = ldf_feature.filter(pl.col("feature_id").is_in(cdm_feat_ids)).select(["feature_id", "start", "end", "type"]).collect()

In [69]:
df_features

feature_id,start,end,type
str,i64,i64,str
"""CDM:feat-7789e6da-eab5-46c7-91…",1,2145,"""CDS"""
"""CDM:feat-8d9e7d7e-6bdc-4bb9-81…",2145,2249,"""CDS"""
"""CDM:feat-7923a96d-9def-47cd-80…",2246,2701,"""CDS"""
"""CDM:feat-ff7fa704-472b-4575-aa…",3,278,"""CDS"""
"""CDM:feat-ad7617c8-9dbe-4004-94…",275,892,"""CDS"""
…,…,…,…
"""CDM:feat-7e1ce321-3940-4130-85…",3725,4522,"""CDS"""
"""CDM:feat-3b38b767-d1ec-4222-a9…",4649,5614,"""CDS"""
"""CDM:feat-9983306d-f095-4b65-b1…",5671,6381,"""CDS"""
